In [2]:
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import re
from random import randint
from time import sleep
import pandas as pd
from google_trans_new import google_translator  
import langid
import numpy as np
import math


# Set up the translator to translate the text from Chinese to English if needed
translator = google_translator()
langid.set_languages(['zh', 'en'])

In [3]:
df = "YOUR DATAFRAME HERE"

# Shoud at least contain FIRT_NAME and LAST_NAME columns

In [5]:
# SET UP THE OPTIONS FOR THE CHROME BROWSER

options = uc.ChromeOptions()
options.add_argument("--window-size=1920,1200")
options.add_argument("--incognito")
#options.add_argument("--headless") # to run the browser in the background if decommented
driver = uc.Chrome(options=options)

username = 'XXX' # your linkedin username
pwd = 'XXX' # your linkedin password

driver.get('https://www.linkedin.com/login/fr?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')
sleep(3)

driver.find_element(By.ID, 'username').send_keys(username)
driver.find_element(By.ID, 'password').send_keys(pwd)

sleep(2)

driver.find_element(By.CSS_SELECTOR, "button[class='btn__primary--large from__button--floating']").click()

sleep(3)

In [ ]:
cols = list([])
final_df = pd.DataFrame(columns=cols)

u = 0
for firstname,name in zip(df['FIRST_NAME'][20:],df['LAST_NAME'][20:]):
        

        full_name = firstname + ' ' + name

        
        try :   
                new_url = "https://www.linkedin.com/search/results/people/?keywords=" + full_name + "&origin=FACETED_SEARCH&sid=pd9"
                driver.get(new_url)

                sleep(3)
                try : 
                        driver.find_element(By.CSS_SELECTOR, "div[class^='entity-result']").find_element(By.CSS_SELECTOR, "a[class^='app-aware-link']").click() 
                except : 
                        continue

        except :        
                driver.get(url)

        sleep(3)

        try : 
                exps = driver.find_element(By.CSS_SELECTOR, "div[class='body']").find_element(By.CSS_SELECTOR, "div[id='experience'] + div + div").find_elements(By.CSS_SELECTOR,"li[class^='artdeco-list']")[0:2]
        except :
                url = "https://www.linkedin.com/search/results/people/?keywords=" + full_name + "&origin=FACETED_SEARCH&schoolFreetext=%22ESSEC%22&sid=pd9"
                driver.get(url)
                sleep(3)
                try : 
                        driver.find_element(By.CSS_SELECTOR, "div[class='entity-result']").find_element(By.CSS_SELECTOR, "a[class^='app-aware-link']").click() 
                except : 
                        continue
                sleep(3)
                
                try : 
                        exps = driver.find_element(By.CSS_SELECTOR, "div[class='body']").find_element(By.CSS_SELECTOR, "div[id='experience'] + div + div").find_elements(By.CSS_SELECTOR,"li[class^='artdeco-list']")[0:2]
                except : 
                        continue
                
        record = {}
        record['last_name'] = name.strip()
        record['first_name'] = firstname.strip()
        i = 1
        for exp in exps :
                
                try :   

                        try : 
                                assert(re.search('[0-9]+.*ans?|[0-9]+.*mois|[0-9]+.*yrs?|[0-9]+.*mos',exp.find_element(By.CSS_SELECTOR, "span:nth-of-type(1) > span").get_attribute("textContent").strip()))
                                company = exp.find_element(By.CSS_SELECTOR, "span:first-of-type").get_attribute("textContent").strip()        
                        except : 
                                company = exp.find_element(By.CSS_SELECTOR, "span:nth-of-type(1) > span").get_attribute("textContent").strip()

                        # if langid.classify(company)[0] == 'zh':
                        #         company = translator.translate(company, dest='en').text.title()
                        # else:
                        #         company = company         
                except :
                        company = ''

                
                try :
                        try :
                                assert(re.search('[0-9]+.*ans?|[0-9]+.*mois|[0-9]+.*yrs?|[0-9]+.*mos',exp.find_element(By.CSS_SELECTOR, "span:nth-of-type(1) > span").get_attribute("textContent").strip()))
                                poste = exp.find_element(By.CSS_SELECTOR,"li:first-of-type").find_element(By.CSS_SELECTOR,"span:nth-of-type(2)").get_attribute('textContent').strip()
                        except : 
                                poste = exp.find_element(By.CSS_SELECTOR, "span:first-of-type > span").get_attribute("textContent").strip()
                        # if langid.classify(poste)[0] == 'zh':
                        #         poste = translator.translate(poste, dest='en').text.title()
                        # else:
                        #         poste = poste
                except : 
                        poste = ''


                try : 
                        try : 
                                assert(re.search('[0-9]+.*ans?|[0-9]+.*mois|[0-9]+.*yrs?|[0-9]+.*mos',exp.find_element(By.CSS_SELECTOR, "span:nth-of-type(1) > span").get_attribute("textContent").strip()))
                                loc = exp.find_element(By.CSS_SELECTOR,"span:nth-of-type(2) > span").get_attribute('textContent').strip()
                        except : 
                                loc = exp.find_element(By.CSS_SELECTOR, "span:nth-child(4) > span").get_attribute("textContent").strip()
                        # if langid.classify(loc)[0] == 'zh':
                        #         loc = translator.translate(loc, dest='en').text.title()
                        # else:
                        #         loc = loc
                except :
                        loc = ''

                if i == 1:

                        record['last_experience'] = company.split('·')[0].strip()
                        record['last_experience_poste'] = poste 
                        record['last_experience_loc'] = loc

                else :
                        record['second_to_last_experience'] = company.split('·')[0].strip()
                        record['second_to_last_experience_poste'] = poste 
                        record['second_to_last_experience_loc'] = loc
                
                i += 1
        
        formation = driver.find_element(By.CSS_SELECTOR, "div[class='body']").find_element(By.CSS_SELECTOR, "div[id='education'] + div + div")

        try :   
                i = 1
                for exp in formation.find_elements(By.CSS_SELECTOR, "li[class^='artdeco-list']"):
                        edu = 'formation_' + str(i)
                        try : 
                                ecole = exp.find_element(By.CSS_SELECTOR, "span:first-of-type").get_attribute("textContent").strip()
                                # if langid.classify(ecole)[0] == 'zh':
                                #         ecole = translator.translate(ecole, dest='en').text.title()
                                # else:
                                #         ecole = ecole
                        except :
                                ecole = ''
                        try : 
                                diplome = exp.find_element(By.CSS_SELECTOR, "span:nth-child(2) > span").get_attribute("textContent").strip()
                                # if langid.classify(diplome)[0] == 'zh':
                                #         diplome = translator.translate(diplome, dest='en').text.title()
                                # else:
                                #         diplome = diplome
                        except : 
                                diplome = ''
                        try : 
                                duree = exp.find_element(By.CSS_SELECTOR, "span:nth-child(3) > span").get_attribute("textContent").strip()
                                # if langid.classify(duree)[0] == 'zh':
                                #         duree = translator.translate(duree, dest='en').text.title()
                                # else:
                                #         duree = duree
                        except :
                                duree = ''
        
                        if len(ecole) == 0 & len(diplome) == 0 & len(duree) == 0 :
                                continue
                        else :
                                record[edu] = ecole + ' | ' + diplome + ' | ' + duree 
                                i += 1

        except :

                print('no formation')

        
        final_df = pd.concat([final_df,pd.DataFrame.from_dict([record])],ignore_index=True)
        final_df.to_csv('2023_MIF_2.csv',index=False)
        print(record)

        u +=1
        if u%10 == 0:
                sleep(20)

In [7]:
driver.quit()

In [11]:

final_df.to_csv('YOUR FINAL DATAFRAME.csv',index=False)